# Evaluation

In [25]:
import pandas as pd

dataset_to_pretty_name = {
    'longeval-heldout-20230513-training': 'June',
    'longeval-long-september-20230513-training': 'Sept.',
    'longeval-short-july-20230513-training': 'July',
}

df = pd.read_json('complete-evaluation.jsonl', lines=True)
df['Dataset'] = df['Dataset'].apply(lambda i: dataset_to_pretty_name.get(i))
df = df[df['name'].str.lower().isin(['bm25', 'bm25-5-variants-prompt-2', 'bm25-10-variants-prompt-2', 'lgd-10-variants-prompt-2', 'pl2-5-variants-prompt-2', 'pl2-10-variants-prompt-2'])]
df[df['Dataset'] == 'Sept.']

,name,Unjudged@10,nDCG@10,condensed-nDCG@10,MRR,Dataset
31,bm25,0.760130,0.183680,0.477587,0.265876,Sept.
36,BM25-5-variants-prompt-2,0.791549,0.164992,0.472690,0.244134,Sept.
37,BM25-10-variants-prompt-2,0.786349,0.169106,0.478501,0.248608,Sept.
55,LGD-10-variants-prompt-2,0.790791,0.166178,0.482670,0.254577,Sept.
60,PL2-5-variants-prompt-2,0.789707,0.164181,0.485604,0.250316,Sept.
61,PL2-10-variants-prompt-2,0.788082,0.166765,0.487811,0.252517,Sept.


# Creation of the Evaluation Data

In [2]:
!pip3 install trectools --no-deps
!pip3 install sarge --no-deps

  Created wheel for trectools: filename=trectools-0.0.49-py3-none-any.whl size=27139 sha256=050c6b04c5007a86a1acf7961858a823f9220cda585ee02520d911c0bb021d21
  Stored in directory: /root/.cache/pip/wheels/6a/cd/17/9a6b28af70445d948c97018b43b9181acd2fdd23e115ee2055
Successfully built trectools
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.


In [12]:
from tira.rest_api_client import Client
import pandas as pd
from tqdm import tqdm
import ir_datasets
import json
from tqdm import tqdm
from trectools import TrecRun, TrecEval, TrecQrel

tira = Client()

TASK = 'ir-benchmarks'
TEAM = 'ows'
    
datasets = ['longeval-heldout-20230513-training', 'longeval-long-september-20230513-training',
            'longeval-short-july-20230513-training', 'longeval-train-20230513-training']


In [5]:
def load_qrels(dataset):
    ds = ir_datasets.load(dataset)
    ds = pd.DataFrame([{"query": i.query_id, "q0": "0", "docid": i.doc_id, "rel": i.relevance} for i in ds.qrels_iter()])
    ret = TrecQrel()
    ret.qrels_data = ds
    
    return ret

QRELS = {
    'longeval-heldout-20230513-training': load_qrels('longeval/heldout'),
    'longeval-long-september-20230513-training': load_qrels('longeval/b-long-september'),
    'longeval-short-july-20230513-training': load_qrels('longeval/a-short-july'),
    'longeval-train-20230513-training': load_qrels('longeval/train'),
}


In [6]:
def report_effectiveness(name, run, dataset):
    te = TrecEval(run, QRELS[dataset])
    
    return {'name': name,
            'Unjudged@10': te.get_unjudged(depth=10),
            'nDCG@10': te.get_ndcg(depth=10, removeUnjudged=False),
            'condensed-nDCG@10': te.get_ndcg(depth=10, removeUnjudged=True),
            'MRR': te.get_reciprocal_rank(removeUnjudged=True),
            'Dataset': dataset,
            }

In [17]:
df = []

for dataset in datasets:
    bm25 = TrecRun(tira.get_run_output(f'{TASK}/{TEAM}/PyTerrier-Index >> BM25', dataset) + '/run.txt')
    df += [report_effectiveness('bm25', bm25, dataset)]
    
    for wmodel in tqdm(['BM25', 'DPH', 'DirichletLM', 'LGD', 'PL2'], f'Evaluation on {dataset}'):
        for prompt in ['1', '2']:
            for variants in ['3', '5', '10']:
                run = TrecRun(f'all-results/{dataset}/{wmodel}-{variants}-variants-prompt-{prompt}-fused-run.txt.gz')
            
                df += [report_effectiveness(f'{wmodel}-{variants}-variants-prompt-{prompt}', run, dataset)]

df = pd.DataFrame(df)

Evaluation on longeval-heldout-20230513-training: 100%|█████████████████████████████████████████████████████████████| 5/5 [00:46<00:00,  9.32s/it]
Evaluation on longeval-long-september-20230513-training: 100%|██████████████████████████████████████████████████████| 5/5 [07:21<00:00, 88.24s/it]
Evaluation on longeval-short-july-20230513-training: 100%|██████████████████████████████████████████████████████████| 5/5 [07:29<00:00, 89.94s/it]
Evaluation on longeval-train-20230513-training: 100%|███████████████████████████████████████████████████████████████| 5/5 [05:24<00:00, 64.86s/it]


In [18]:
df.to_json('complete-evaluation.jsonl', lines=True, orient='records')

In [19]:
!ls -lh complete-evaluation.jsonl

-rw-rw-r--    1 root     users      18.6K Jun  6 09:21 complete-evaluation.jsonl
